In [ ]:
%%capture
!pip install torchmetrics wandb

In [ ]:
# Загрузка датасета
%%capture
!gdown 1UNqS5EORC5r38PY8OgoKvzLPQMBzXhRh
!unzip "R_peaks_train.zip"

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as ptl
import torchmetrics
from tqdm.auto import tqdm
from IPython.display import clear_output

import torch
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import models

from google.colab import drive
drive.mount('/content/drive')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

Mounted at /content/drive


device(type='cuda')

In [ ]:
!wandb login 20f90c141c14e3f8e2d80816e21a0960cf446951
import wandb


wandb.init(
    project="AIIJC",

    config={
        "architecture": "efficientnet_v2",
        "dataset": "R_peaks",
    }
)

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: Currently logged in as: notdiff. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.15.10
wandb: Run data is saved locally in /content/wandb/run-20230919_200343-163nlg4b
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run fine-wave-108
wandb: ⭐️ View project at https://wandb.ai/notdiff/AIIJC
wandb: 🚀 View run at https://wandb.ai/notdiff/AIIJC/runs/163nlg4b


In [ ]:
data = pd.read_csv('/content/R_peaks_train/labels.csv')

normals = data[data.myocard == 0][:6000]
myocard = data[data.myocard == 1]

data = pd.concat([normals, myocard]).sample(frac = 1)

In [ ]:
class SignalsDataset(Dataset):
    def __init__(self, labels, path='/'):
        self.x_paths = [labels.iloc[i, 0] for i in range(len(labels)) if labels.iloc[i, 0] not in ['09337_hr_0', '17552_hr_0']]
        self.labels = [labels.iloc[i, 1] for i in range(len(labels)) if labels.iloc[i, 0] not in ['09337_hr_0', '17552_hr_0']]
        self.path = path

    def __len__(self):
        return len(self.x_paths)

    def __getitem__(self, idx):
        hr = torch.tensor(np.load(self.path + self.x_paths[idx] + '.npy'))[None, :, :]

        target = self.labels[idx]

        return hr, target

In [ ]:
dataset = SignalsDataset(data, '/content/R_peaks_train/signals/')
train_data, val_data = random_split(dataset, [0.9, 0.1])

batch_size = 64

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)

In [ ]:
model = models.efficientnet_v2_s(pretrained=True)


model.classifier[1] = torch.nn.Linear(in_features=model.classifier[1].in_features, out_features=2, bias=True)

model.features[0][0] = torch.nn.Conv2d(1, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)

for p in model.parameters():
    p.requires_grad = True

model.to(device)
pass

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_S_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_v2_s-dd5fe13b.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_s-dd5fe13b.pth
100%|██████████| 82.7M/82.7M [00:54<00:00, 1.58MB/s]


In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()

learning_rate = 3e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)

In [ ]:
def train(model, loss_fn, scheduler, optimizer, n_epoch=3, device='cuda'):
    train_losses = []
    val_losses = []
    train_acc = []
    val_acc = []
    val_f1 = []

    max_f1 = 0

    val_accuracy, val_loss, val_f1_score = evaluate(model, val_loader, loss_fn=loss_fn, device=device)
    wandb.log({"F1": val_f1_score, "Acc": val_accuracy, 'loss': val_loss})

    for epoch in range(n_epoch):
        print("Epoch:", epoch+1)



        model = model.train()
        for batch in tqdm(train_loader):
            X_batch, y_batch = batch
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            logits = model(X_batch.float())

            loss = loss_fn(logits, y_batch.to(torch.int64))

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            train_losses.append(loss.item())

            model_answers = torch.argmax(logits, dim=1)
            train_accuracy = torch.sum(y_batch == model_answers) / len(y_batch)
            train_acc.append(train_accuracy.item())


        model.eval()

        val_accuracy, val_loss, val_f1_score = evaluate(model, val_loader, loss_fn=loss_fn, device=device)
        wandb.log({"F1": val_f1_score, "Acc": val_accuracy, 'loss': val_loss})
        clear_output(wait=True)

        if max_f1 < val_f1_score:
            max_f1 = val_f1_score
            torch.save(model.state_dict(), f'/content/drive/MyDrive/models/eff_v2_s_Rp_resampled_model_f1:{val_f1_score:.3f}_6000.pth')


        val_losses.append(val_loss.item())
        val_acc.append(val_accuracy)

        scheduler.step(val_loss)

def evaluate(model, dataloader, loss_fn, device):
    losses = []
    num_correct = 0
    num_elements = 0
    f1 = torchmetrics.F1Score(task='binary')
    f1_score = 0

    model = model.eval()
    for batch in tqdm(dataloader):
        X_batch, y_batch = batch
        X_batch, y_batch = X_batch.to(device), y_batch.to(device).float()

        with torch.no_grad():
            logits = model(X_batch.float())

            loss = loss_fn(logits, y_batch.to(torch.int64))
            losses.append(loss.item())

            y_pred = torch.argmax(logits, dim=1)

            f1_score += f1(y_pred.cpu(), y_batch.cpu())

            num_elements += len(y_batch)
            num_correct += torch.sum(y_pred == y_batch)

    accuracy = num_correct / num_elements
    f1_score = f1_score / len(dataloader)

    return accuracy.item(), np.mean(losses), f1_score.item()

In [ ]:
train(model, loss_fn, scheduler, optimizer, n_epoch=40, device=device)

Epoch: 40


  0%|          | 0/144 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]